# Running the Classifier



## Setting up the classes

In [ ]:
!pip install dcm-classifier0.6.0rc7

In [4]:
from dcm_classifier.study_processing import ProcessOneDicomStudyToVolumesMappingBase
from dcm_classifier.image_type_inference import ImageTypeClassifierBase
from pathlib import Path


current_directory: Path = Path.cwd()
root_directory = current_directory.parent

session_directory = root_directory / "tests" / "testing_data" / "anonymized_testing_data" / "anonymized_data"

model = root_directory / "models" / "rf_classifier.onnx"

inferer = ImageTypeClassifierBase(classification_model_filename=model.as_posix())
study = ProcessOneDicomStudyToVolumesMappingBase(
    study_directory=session_directory.as_posix(), inferer=inferer
)

study.run_inference()

The directory: /home/cavriley/programs/dcm-classifier/tests/testing_data/anonymized_testing_data/anonymized_data contains 16 DICOM sub-volumes


InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Got invalid dimensions for input: float_input for the following indices
 index: 1 Got: 48 Expected: 29
 Please fix either the inputs or the model.

## Choosing preferred files

### Saving to Nifti

Saving to a nifti file can be done in two ways here, ITK and dcm2niix both provide methods however the ITK method is preferred because of its robustness. For these examples, we are looking for T1W modality and axial acquisition plane images. 


#### Using ITK

Once the image is obtained, the image can easily be saved using ITK. ITK is the preferred method for saving the image as it is more robust in the sense that many image types can be handled including .nii.gz and .nrrd files unlike dcm2niix.

In [ ]:
import itk

for series_number, series in study.series_dictionary.items():
    for index, volume in enumerate(series.get_volume_list()):
        if volume.get_volume_modality() == "t1w" and volume.get_acquisition_plane() == "ax":
            volume_image = volume.get_itk_image()
            itk.imwrite(volume_image, "tutorial_itk_image.nii.gz")

#### Using dcm2niix

The dcm2niix method is also available, however it is not as robust as the ITK method. The dcm2niix method is used to convert the DICOM files to a nifti  and does not support other file types such as .nrrd. Also, the dcm2niix method is an external method and requires the dcm2niix software to be installed on the system.

In [ ]:
from subprocess import run

def create_nifti_file(
    dcm2niix_path: str,
    subject: str,
    session: str,
    modality: str,
    plane: str,
    output_dir: str,
    dcm_dir: str,
    desc: str = None,
) -> None:
    run_num = 1
    if desc is not None:
        fname = (
            f"{subject}_{session}_run-00{run_num}_acq-{plane}_desc-{desc}_{modality}"
        )
    else:
        fname = f"{subject}_{session}_run-00{run_num}_acq-{plane}_{modality}"
    while Path(f"{output_dir}/{fname}.nii.gz").exists():
        fname = fname.replace(f"run-00{run_num}", f"run-00{run_num+1}")
        run_num += 1
    run(
        [dcm2niix_path, "-o", output_dir, "-f", fname, "-z", "y", dcm_dir]
    ) 
    print(f"{dcm2niix_path} -o {output_dir} -f {fname} -n -z y {dcm_dir}")

The DICOM files are moved to a temporary directory because the files should all be located within the same directory for dcm2niix to work.

In [ ]:
import tempfile
import shutil
    
dcm2niix_path = "FILL IN"

sub = "sub-01-test"
ses = "ses-01-test"

final_dir_path = "FILL IN""

for series_number, series in study.series_dictionary.items():
    for index, volume in enumerate(series.get_volume_list()):
        if volume.get_volume_modality() == "t1w" and volume.get_acquisition_plane() == "ax":
            temp_dir = tempfile.mkdtemp()
            file_list = volume.get_one_volume_dcm_filenames()
            for file in file_list:
                shutil.copy(file, temp_dir)
            create_nifti_file(
                dcm2niix_path=dcm2niix_path,
                subject=sub,
                session=ses,
                modality=volume.get_volume_modality(),
                plane=volume.get_acquisition_plane(),
                output_dir=final_dir_path,
                dcm_dir=temp_dir,
            )